In [1]:
import os
%cd ../../..
print(os.getcwd())

/Users/daniel.vella/Desktop/dv-box/msc/deepchem
/Users/daniel.vella/Desktop/dv-box/msc/deepchem


In [88]:
"""
Train low-data siamese models on MUV. Test last fold only.
"""
from __future__ import print_function
from __future__ import division
from __future__ import unicode_literals

import numpy as np
import deepchem as dc
import tensorflow as tf
# from examples.low_data.datasets import load_muv_convmol
import copy

In [89]:
# Number of folds for split 
K = 4
# num positive/negative ligands
n_pos = 10
n_neg = 10
# Set batch sizes for network
test_batch_size = 128
support_batch_size = n_pos + n_neg
nb_epochs = 1
n_train_trials = 2000
n_eval_trials = 20
n_steps_per_trial = 1
learning_rate = 1e-4
log_every_n_samples = 50
# Number of features on conv-mols
n_feat = 75

tasks, datasets, transformers = dc.molnet.load_muv(featurizer='ECFP', split='random')
# tasks_copy, datasets_copy, transformers_copy = copy.deepcopy(tasks), copy.deepcopy(datasets), copy.deepcopy(transformers)

In [90]:
train_dataset, valid_dataset, test_dataset = datasets

In [91]:
# X contains 74469 samples with fingerprint of length 1024
# y represents 17 tasks of the MUV tasks
# w represents weights
print(train_dataset)

<DiskDataset X.shape: (74469, 1024), y.shape: (74469, 17), w.shape: (74469, 17), task_names: ['MUV-466' 'MUV-548' 'MUV-600' ... 'MUV-852' 'MUV-858' 'MUV-859']>


In [119]:
keras_model = tf.keras.Sequential([
    tf.keras.layers.Dense(1000, activation='tanh'),
    tf.keras.layers.Dense(1)
])

model = dc.models.KerasModel(keras_model,
                             loss=dc.models.losses.L2Loss())


In [120]:
model.fit(train_dataset,
         nb_epoch=1)

0.0008366763591766358

In [121]:
len(train_dataset.y.shape)

2

In [122]:
# PRC AUC recommended for MUV
metric = dc.metrics.Metric(dc.metrics.prc_auc_score, mode="classification")

In [123]:
print("Training Score: ", model.evaluate(train_dataset, metric, transformers, n_classes=17, use_sample_weights=True))
print("Test Score: ", model.evaluate(test_dataset, metric, transformers))

ValueError: Shape of input doesn't match expected n_tasks=17

In [ ]:
# task_splitter = dc.splits.TaskSplitter()
# fold_datasets = task_splitter.k_fold_split(dataset, K)

# train_folds = fold_datasets[:-1]
# train_dataset = dc.splits.merge_fold_datasets(train_folds)
# test_dataset = fold_datasets[-1]

# Train support model on train
support_model = dc.nn.SequentialSupportGraph(n_feat)

# Add layers
support_model.add(dc.nn.GraphConv(64, n_feat, activation='relu'))
support_model.add(dc.nn.GraphPool())
support_model.add(dc.nn.GraphConv(128, 64, activation='relu'))
support_model.add(dc.nn.GraphPool())
support_model.add(dc.nn.GraphConv(64, 128, activation='relu'))
support_model.add(dc.nn.GraphPool())
support_model.add(dc.nn.Dense(128, 64, activation='tanh'))

support_model.add_test(dc.nn.GraphGather(test_batch_size, activation='tanh'))
support_model.add_support(
    dc.nn.GraphGather(support_batch_size, activation='tanh'))

model = dc.models.SupportGraphClassifier(
    support_model,
    test_batch_size=test_batch_size,
    support_batch_size=support_batch_size,
    learning_rate=learning_rate)

model.fit(
    train_dataset,
    nb_epochs=nb_epochs,
    n_episodes_per_epoch=n_train_trials,
    n_pos=n_pos,
    n_neg=n_neg,
    log_every_n_samples=log_every_n_samples)
mean_scores, std_scores = model.evaluate(
    test_dataset, metric, n_pos, n_neg, n_trials=n_eval_trials)

print("Mean Scores on evaluation dataset")
print(mean_scores)
print("Standard Deviations on evaluation dataset")
print(std_scores)
print("Median of Mean Scores")
print(np.median(np.array(mean_scores.values())))
